In [14]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import undetected_chromedriver as uc
import time
import random


SCRAPING PISO

In [15]:
# Abrir una ventana de incógnito
options = uc.Chrome()
#options.add_argument("--incognito")
browser = uc.Chrome()
url = "https://www.idealista.com/inmueble/102911174/"
browser.get(url)
browser.find_element('id', 'didomi-notice-disagree-button').click()

In [84]:
html = browser.page_source
soup = bs(html, 'html.parser')

In [85]:
seccion_caracteristicas = soup.find('section', {'id' : 'details'})
seccion_ubicacion = soup.find('div', {'id' : 'headerMap'})

In [89]:
caracteristicas = [caract.text.strip() for caract in seccion_caracteristicas.find_all('li')]
ubicacion = [caract.text.strip() for caract in seccion_ubicacion.find_all('li')][0:2]

In [91]:
ubicacion

['Avenida de França', 'Barrio Penya-Roja']

SCRAPING VALENCIA

In [77]:
browser = uc.Chrome()
url_valencia = 'https://www.idealista.com/venta-viviendas/valencia-valencia/'
browser.get(url_valencia)
browser.find_element('id', 'didomi-notice-disagree-button').click()

In [78]:
html = browser.page_source
soup = bs(html, 'html')

In [79]:
#SECCION DE PISOS
seccion_inmuebles = soup.find('section', {'class' : 'items-container items-list'})

In [80]:
#Obtener Id's de los inmuebles
articles = seccion_inmuebles.find_all('article')
id_inmuebles = [article.get('data-element-id') for article in articles]

In [81]:
#Limpiar nulos
id_inmuebles = [inmueble for inmueble in id_inmuebles if inmueble is not None]

BUSQUEDA INMUEBLES

In [ ]:
x = 1
ids = []
browser = uc.Chrome()
while True:
    
    url = f'https://www.idealista.com/venta-viviendas/valencia-valencia/pagina-{x}.htm'
    
    browser.get(url)
    time.sleep(random.randint(10,12))
    
    try:
        browser.find_element('id', 'didomi-notice-disagree-button').click()
    except:
        pass
    
    html = browser.page_source
    soup = bs(html, 'html')
    
    pag_seleccionada = int(soup.find('div', {'class' : 'pagination'}).find('li', {'class' : 'selected'}).find('span').text)
    
    if x == pag_seleccionada:
        articles = soup.find('main', {'class' : 'listing-items'}).find_all('article')
    else:
        break
    
    x += 1
    
    for article in articles:
        id_inmuebles = article.get('data-element-id')
        
        ids.append(id_inmuebles)
        
        time.sleep(random.randint(1,3))
        
    ids = [inmueble for inmueble in ids if inmueble is not None]

In [74]:
ids_inmuebles = pd.DataFrame(ids)
ids_inmuebles.columns = ['id']
ids_inmuebles.to_csv('ids_inmuebles.csv', index = False)

In [16]:
inmuebles = pd.Series()

In [18]:
# Diccionario con todas las posibles características

def parse_inmueble(id_inmueble):
    url = f'https://www.idealista.com/inmueble/{id_inmueble}/'
    browser.get(url)
    
    html = browser.page_source
    soup = bs(html, 'html')
    
    #AÑADIR ID INMUEBLE
    df_inmueble = pd.DataFrame({'id_inmueble': [id_inmueble]})
    
    seccion_caracteristicas = soup.find('section', {'id' : 'details'})
    try:
        caracteristicas_inmueble =  [caract.text.strip() for caract in seccion_caracteristicas.find_all('li')]
    except:
        return
    
    # Diccionario con todas las posibles características
    caracteristicas_dict = {
        'Superficie construida': None,
        'Superficie útil': None,
        'Habitaciones': None,
        'Baños': None,
        'Terraza': None,
        'Balcón': None,
        'Plaza de garaje': None,
        'Estado': None,
        'Armarios empotrados': None,
        'Calefacción': None,
        'Orientación': None,
    }
    ubicacion_dict = {
        'Barrio': None,
        'Distrito': None
    }
    precio_dict = {
        'Precio': None
    }
    
    #CARACTERISTICAS
    for caracteristica in caracteristicas_inmueble:
        if ':' in caracteristica:
            key, value = caracteristica.split(':', 1)
            key = key.strip()
            value = value.strip()
            if key in caracteristicas_dict:
                caracteristicas_dict[key] = value
            else:
                caracteristicas_dict[key] = value
        else:
            # Buscar superficie construida y útil sin clave-valor
            if 'construidos' in caracteristica.lower():
                superficie_construida = caracteristica.split(',')[0].strip()
                superficie_construida = superficie_construida.replace('m² construidos', '').strip()
                caracteristicas_dict['Superficie construida'] = superficie_construida
            if 'útiles' in caracteristica.lower():
                superficie_util = caracteristica.split(',')[1].strip()
                superficie_util = superficie_util.replace('m² útiles', '').strip()
                caracteristicas_dict['Superficie útil'] = superficie_util
            
            for key in caracteristicas_dict:
                if key.lower() in caracteristica.lower():
                    if caracteristicas_dict[key] is None:
                        caracteristicas_dict[key] = caracteristica
                    else:
                        caracteristicas_dict[key] += ', ' + caracteristica
    
    #UBICACION
    seccion_ubicacion = soup.find('div', {'id' : 'headerMap'})
    caracteristicas_ubicacion = [caract.text.strip() for caract in seccion_ubicacion.find_all('li')][1:3]
    
    for ubicacion in caracteristicas_ubicacion:
        if ':' in ubicacion:
            key, value = ubicacion.split(':', 1)
            key = key.strip()
            value = value.strip()
            if key in ubicacion_dict:
                ubicacion_dict[key] = value
        else:
            for key in ubicacion_dict:
                if key.lower() in ubicacion.lower():
                    ubicacion_dict[key] = ubicacion
                    break
            else:
                ubicacion_dict[len(ubicacion_dict) + 1] = ubicacion

    caracteristicas_dict.update(ubicacion_dict)
    
    #PRECIO    
    seccion_precio = soup.find('div', {'class' : 'info-data'})
    precio = seccion_precio.find('span', {'class' : 'txt-bold'}).text.replace('.','')
    precio_dict['Precio'] = precio
    
    caracteristicas_dict.update(precio_dict)
    
    df_inmueble = df_inmueble.join(pd.DataFrame([caracteristicas_dict]))
    
    return df_inmueble

In [19]:
df = pd.read_csv('ids_inmuebles_full.csv')
ids = df.values.flatten()

df_casas = parse_inmueble(ids[0])


for i in range(1, len(ids)):
    df_casas = pd.concat([df_casas, parse_inmueble(ids[i])])
    
    time.sleep(random.randint(4,8))

In [20]:
df_casas.to_csv('datos_full.csv', index = False)